In [65]:
import numpy as np

class Reversy():
    def __init__(self):
        self.map = []
        self.turn = 0
        self.mark = ['O','X']
        self.able = []
        self.checker = False
        self.notend = True
        self.log = []
        self.able_count = 0
    
    def reset(self):
        self.map = [
            [-1,-1,-1,-1,-1,-1,-1,-1],
            [-1,-1,-1,-1,-1,-1,-1,-1],
            [-1,-1,-1,-1,-1,-1,-1,-1],
            [-1,-1,-1,0,1,-1,-1,-1],
            [-1,-1,-1,1,0,-1,-1,-1],
            [-1,-1,-1,-1,-1,-1,-1,-1],
            [-1,-1,-1,-1,-1,-1,-1,-1],
            [-1,-1,-1,-1,-1,-1,-1,-1]
        ]
        self.turn = 0
        self.able_list()
        self.checker = False
        self.notend = True
        self.log = []

        return self.able

    def sample(self):
        return self.able[np.random.choice(self.able_count)]

    def able_list(self):
        self.able = []
        self.able_count = 0
        for i in range(8):
            for j in range(8):
                if self.find_next(i+1,j,6,False) + self.find_next(i-1,j,4,False) + self.find_next(i,j-1,2,False) + self.find_next(i,j+1,8,False) + self.find_next(i+1,j-1,3,False) + self.find_next(i-1,j+1,7,False) + self.find_next(i-1,j-1,1,False) + self.find_next(i+1,j+1,9,False):
                    if self.map[i][j] == -1:
                        self.able.append([i,j])
                        self.able_count += 1

        if len(self.able) == 0:
            self.notend = False
        else:
            self.checker = False

    def step(self, action):
        x, y = action

        self.turn += 1
        self.map[x][y] = (self.turn % 2)

        self.log.append([x,y])

        self.reverse(x-1,y-1,1,0)
        self.reverse(x,y-1,2,0)
        self.reverse(x+1,y-1,3,0)
        self.reverse(x-1,y,4,0)
        self.reverse(x+1,y,6,0)
        self.reverse(x-1,y+1,7,0)
        self.reverse(x,y+1,8,0)
        self.reverse(x+1,y+1,9,0)

        return action, self.count_score(), not self.notend, {"prob": 1/max(self.able_count, 0.00000001)}, None

    def count_score(self):
        score = [0,0]

        for i in range(8):
            for j in range(8):
                if self.map[i][j] < 0:
                    continue
                score[self.map[i][j]] += 1

        if self.turn:
            return score[1] - score[0]
        return score[0] - score[1]

    def reverse(self,x,y,seed,cnt):
        if x == -1 or y == -1 or x == 8 or y == 8 or self.map[x][y] == -1:
            return
        if self.map[x][y] != (self.turn % 2):
            if seed == 1:
                self.reverse(x-1,y-1,seed,cnt+1)
            if seed == 2:
                self.reverse(x,y-1,seed,cnt+1)
            if seed == 3:
                self.reverse(x+1,y-1,seed,cnt+1)
            if seed == 4:
                self.reverse(x-1,y,seed,cnt+1)
            if seed == 6:
                self.reverse(x+1,y,seed,cnt+1)
            if seed == 7:
                self.reverse(x-1,y+1,seed,cnt+1)
            if seed == 8:
                self.reverse(x,y+1,seed,cnt+1)
            if seed == 9:
                self.reverse(x+1,y+1,seed,cnt+1)
        else:
            if cnt:
                if seed == 1:
                    for i in range(1,cnt+1):
                        self.map[x+i][y+i] = (self.turn % 2)
                if seed == 2:
                    for i in range(1,cnt+1):
                        self.map[x][y+i] = (self.turn % 2)
                if seed == 3:
                    for i in range(1,cnt+1):
                        self.map[x-i][y+i] = (self.turn % 2)
                if seed == 4:
                    for i in range(1,cnt+1):
                        self.map[x+i][y] = (self.turn % 2)
                if seed == 6:
                    for i in range(1,cnt+1):
                        self.map[x-i][y] = (self.turn % 2)
                if seed == 7:
                    for i in range(1,cnt+1):
                        self.map[x+i][y-i] = (self.turn % 2)
                if seed == 8:
                    for i in range(1,cnt+1):
                        self.map[x][y-i] = (self.turn % 2)
                if seed == 9:
                    for i in range(1,cnt+1):
                        self.map[x-i][y-i] = (self.turn % 2)

    def find_next(self,x,y,seed,bool):
        if x == -1 or y == -1 or x == 8 or y == 8 or self.map[x][y] == -1:
            return 0
        if self.map[x][y] == (self.turn % 2):
            if seed == 1:
                return self.find_next(x-1,y-1,seed,True)
            if seed == 2:
                return self.find_next(x,y-1,seed,True)
            if seed == 3:
                return self.find_next(x+1,y-1,seed,True)
            if seed == 4:
                return self.find_next(x-1,y,seed,True)
            if seed == 6:
                return self.find_next(x+1,y,seed,True)
            if seed == 7:
                return self.find_next(x-1,y+1,seed,True)
            if seed == 8:
                return self.find_next(x,y+1,seed,True)
            if seed == 9:
                return self.find_next(x+1,y+1,seed,True)
        else:
            return bool

In [66]:
env = Reversy()
env.reset()
episodes = 10
gamma = 0.99

In [67]:
import random
import numpy as np
from tqdm import *

# 감가 계산 알고리즘
def decay_schedule(init_value, min_value, decay_ratio, max_steps, log_start=-2, log_base=10):
    decay_steps = int(max_steps * decay_ratio)
    rem_steps = max_steps - decay_steps
    values = np.logspace(log_start, 0, decay_steps, base=log_base, endpoint=True)[::-1]
    values = (values - values.min()) / (values.max() - values.min())
    values = (init_value - min_value) * values + min_value
    values = np.pad(values, (0, rem_steps), "edge")

    return values

# Q-학습(Q-Learning) 알고리즘
def q_learning(env, gamma=1.0,
               init_alpha=0.5, min_alpha=0.01, alpha_decay_ratio=0.5,
               init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
               n_episodes=3000):
    
    Q = np.zeros((8, 8), dtype=np.float64)
    Q_track = np.zeros((n_episodes, 8, 8), dtype=np.float64)
    pi_track = []
    
    alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)
    epsilons = decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)

    for e in tqdm(range(n_episodes), leave=False):
        env.reset()
        done = False

        while not done:
            action = env.sample()
            next_state, reward, done, _, _ = env.step(action)
            td_target = reward + gamma * Q[next_state].max() * (not done) # SarSa와 다른 점
            td_error = td_target - Q[state][action]
            Q[state][action] = Q[state][action] + alphas[e] * td_error
            state = next_state
        
        Q_track[e] = Q
        pi_track.append(np.argmax(Q, axis=1))

    V = np.max(Q, axis=1)
    pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]

    return Q, V, pi, Q_track, pi_track


Q_qls = []
V_qls = []
Q_track_qls = []

SEEDS = (12, 34)

for seed in tqdm(SEEDS, desc="All seeds", leave=True):
    random.seed(seed)
    np.random.seed(seed)
    env.reset()

    Q_ql, V_ql, pi_ql, Q_track_ql, pi_track_ql = q_learning(env, gamma=gamma, n_episodes=episodes)
    Q_qls.append(Q_ql)
    V_qls.append(V_ql)
    Q_track_qls.append(Q_track_ql)

Q_ql = np.mean(Q_qls, axis=0)
V_ql = np.mean(V_qls, axis=0)
Q_track_ql = np.mean(Q_track_qls, axis=0)

All seeds:   0%|          | 0/2 [00:00<?, ?it/s]


UnboundLocalError: cannot access local variable 'state' where it is not associated with a value

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])